In [6]:
import os
import requests
import pandas as pd
import json
import re
import zipfile
import numpy as np
from bs4 import BeautifulSoup
import urllib

In [ ]:
# Directories
outdir = "outputs/consumption_and_expenditures"

In [ ]:
# For consumption and expenditures URL formula: base_url + year + consumption_url_ending
base_url = "https://www.eia.gov/consumption/residential/data/"
consumption_url_ending = "/index.php?view=consumption"

In [ ]:
years = [1993, 1997, 2001, 2005, 2009, 2015, 2020]

for year in years:

   year_dir = os.path.join(outdir, str(year))
   if not os.path.exists(year_dir):
      os.mkdir(year_dir)

   curr_url = base_url + str(year) + consumption_url_ending
   resp = requests.get(curr_url)

   if (resp.status_code == 200):
      soup = BeautifulSoup(resp.content)
      links = soup.find_all('a', href = True)
      links = [x['href'] for x in links if '.xls' in x['href'] or ".pdf" in x['href']]

      for link in links:

         if "/consumption/residential/data" in link:
            link = link[len("/consumption/residential/data/1993/"):]

         link_full_url = base_url + str(year) + "/" + link
         link_resp = requests.get(link_full_url)
         test = re.findall("^.*\/(.*)$", link)
         table_name = re.findall("^.*\/(.*)$", link)[0]
         print(f'{link_full_url} {link_resp.status_code} {table_name}')

         if (link_resp.status_code == 200):
            output = open(os.path.join(year_dir, table_name), 'wb')
            output.write(link_resp.content)
            output.close()


In [ ]:
# https://www.eia.gov/consumption/residential/data/2015/c&e/pdf/ce1.1.pdf